In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda

In [5]:
training_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라미터

* 에폭수 : 데이터셋을 반복하는 횟수 
* 배치크기 : 매개변수가 갱신되기 전 신경망을 통해 전파도니 데이터 샘플의 수 
* 학습률 : 각 배치/에폭에서 모델의 매개변수를 조절하는 비율, 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.

In [10]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 손실함수

손실함수는 

In [11]:
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측과 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nn Accuracty: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 2.296379 [    0/60000]
loss: 2.284052 [ 6400/60000]
loss: 2.264962 [12800/60000]
loss: 2.266000 [19200/60000]
loss: 2.241317 [25600/60000]
loss: 2.209397 [32000/60000]
loss: 2.219442 [38400/60000]
loss: 2.177205 [44800/60000]
loss: 2.181363 [51200/60000]
loss: 2.142234 [57600/60000]
Test Error: 
n Accuracty: 41.6%, Avg loss: 2.142223 

Epoch 2
--------------------
loss: 2.150153 [    0/60000]
loss: 2.134759 [ 6400/60000]
loss: 2.071650 [12800/60000]
loss: 2.101054 [19200/60000]
loss: 2.033425 [25600/60000]
loss: 1.972865 [32000/60000]
loss: 2.010910 [38400/60000]
loss: 1.912819 [44800/60000]
loss: 1.937693 [51200/60000]
loss: 1.850625 [57600/60000]
Test Error: 
n Accuracty: 51.6%, Avg loss: 1.852288 

Epoch 3
--------------------
loss: 1.886868 [    0/60000]
loss: 1.847301 [ 6400/60000]
loss: 1.723688 [12800/60000]
loss: 1.783170 [19200/60000]
loss: 1.661966 [25600/60000]
loss: 1.620793 [32000/60000]
loss: 1.653877 [38400/60000]
loss: 1.543383 [44800/